<h1 align='center'>Incorporando um modelo de aprendizado de máquina em um aplicativo da Web</h1>

<p align='center'><img src=https://afsystems.com.br/wp-content/uploads/2019/03/20190327_voce_sabe_o_que_e_aplicavitos_web_.jpg width='500'></p>

As técnicas de aprendizado de máquina não se limitam a aplicativos e análises *offline*, e se tornaram o mecanismo preditivo de vários serviços da *web*. Por exemplo, aplicativos populares e úteis de modelos de aprendizado de máquina em aplicativos da *Web* incluem detecção de *spam* em formulários de envio, mecanismos de pesquisa, sistemas de recomendação para mídia ou portais de compras e muito mais.

### Serializando estimadores scikit-learn ajustados

Treinar um modelo de aprendizado de máquina pode ser **computacionalmente caro**. Certamente, não queremos treinar novamente nosso modelo toda vez que fechamos nosso interpretador *Python* e queremos fazer uma nova previsão ou recarregar nosso aplicativo da web.

Uma opção para persistência de modelo é o módulo *pickle* embutido do *Python* (https://docs.python.org/3.7/library/pickle.html), que nos permite serializar e desserializar estruturas de objeto *Python* para compactar *bytecode* para que possamos salvar nosso classificador em seu estado atual e recarregá-lo se quisermos classificar novos exemplos não rotulados, sem precisar que o modelo aprenda com os dados de treinamento novamente.

Antes de executar o código a seguir, certifique-se de ter treinado um modelo de regressão logística *out-of-core* e de tê-lo pronto em sua sessão atual do *Python*: